In [2]:
%pylab inline

import os,sys
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.gridspec as gridspec

from collections import defaultdict, Counter, OrderedDict

import cytograph as cg
import loompy
import palettable

sys.path.append(os.path.realpath(os.path.join(os.getcwd(), '..', '..')))
from scbeta_scrnaseq import utils
from scbeta_scrnaseq import vis
import scbeta_scrnaseq.cytograph_inmem_utils as cgm
import scbeta_scrnaseq.cytograph_analyses as cga



2019-02-20 13:55:45,357 DEBUG: Loaded backend module://ipykernel.pylab.backend_inline version unknown.


Populating the interactive namespace from numpy and matplotlib


### Extract progenitors cells from day 0 and day 1

In [ ]:
# full_ds = loompy.connect(f'../data/complete_processing/stage5.processed.loom', 'r')
# full_ds.vals = sp.sparse.csr_matrix(full_ds.layers[""][:, :])


In [ ]:
%mkdir ../data/pseudotime/
attributes_to_copy = ['CellID', 'CellBatch', 'CellDay', 'CellFlask', 'DetailedLabels', 'Labels', '_TrainFilter', '_Valid']

f_clusters = np.isin(full_ds.ca.Labels, ['exo', 'prog_sox2', 'prog_nkx61', 'neurog3_early', 'neurog3_late', 'neurog3_mid'])
# f_clusters = np.isin(full_ds.ca.Labels, ['exo', 'prog_sox2', 'prog_nkx61', 'neurog3_early', ])
f_time = (full_ds.ca.CellDay == 0)

f_cells = np.where((f_time & f_clusters))[0]
min_cells_per_valid_gene = 10
loompy.create(f'../data/pseudotime/stage5.day0.loom',
              {'': full_ds.vals[:, f_cells]},
              {'Gene': full_ds.ra.Gene,
               '_Valid': (full_ds.vals[:, f_cells].sum(1).A.ravel() > min_cells_per_valid_gene),
               '_Regress': (full_ds.vals[:, f_cells].sum(1).A.ravel() > min_cells_per_valid_gene),
                  },
              {a: full_ds.ca[a][f_cells] for a in attributes_to_copy},
             )

In [ ]:
attributes_to_copy = ['CellID', 'CellBatch', 'CellDay', 'CellFlask', 'DetailedLabels', 'Labels', '_TrainFilter', '_Valid']

f_clusters = np.isin(full_ds.ca.Labels, ['exo', 'prog_sox2', 'prog_nkx61', 'neurog3_early', 'neurog3_late', 'neurog3_mid'])
f_time = (full_ds.ca.CellDay == 1)

f_cells = np.where((f_time & f_clusters))[0]
min_cells_per_valid_gene = 10
loompy.create(f'../data/pseudotime/stage5.day1.loom',
              {'': full_ds.vals[:, f_cells]},
              {'Gene': full_ds.ra.Gene,
               '_Valid': (full_ds.vals[:, f_cells].sum(1).A.ravel() > min_cells_per_valid_gene),
               '_Regress': (full_ds.vals[:, f_cells].sum(1).A.ravel() > min_cells_per_valid_gene),
                  },
              {a: full_ds.ca[a][f_cells] for a in attributes_to_copy},
             )
full_ds.close()

In [ ]:
import json
with open('../../01_Stages_3_to_6/data/original/cell_cycle_genes.json', 'r') as f:
    cell_cycle_set = set(json.load(f))
     

In [ ]:
seed = 91829211
tds = {}
for tp in range(2):
    tds[tp] = loompy.connect(f'../data/pseudotime/stage5.day{tp}.loom')
    tds[tp].vals = sp.sparse.csr_matrix(tds[tp].layers[""][:, :])
    _tds = tds[tp]
    
    _valid_genes = _tds.ra._Valid
    _valid_genes[np.where(np.isin(_tds.ra.Gene, list(cell_cycle_set)))[0]] = 0
    _tds.ra._Valid = _valid_genes
    
    cga.highvar_pca(_tds, _tds.vals, namespace='', seed=seed,
                train_cells=np.where(_tds.ca['_TrainFilter'] > 0)[0],
                n_highvar_pcs=50,
                n_highvar_genes=2000)
    
    tds.ca["HighVarTSNE"] = cg.TSNE(perplexity=100).layout(tds.ca[r"HighVarPCA"][:, :10])

### Computation of pseudotime

In [10]:
time_alignment_vector = {0: pd.Series((tds[0].ca.Labels!='prog_nkx61').astype(int), tds[0].ca.CellID),
                 1: pd.Series((tds[1].ca.Labels!='neurog3_early').astype(int), tds[1].ca.CellID),
                 }

for tp in range(2):
    _tds = tds[tp]
    import scanpy.api as sc
    adata = sc.AnnData(_tds.ca.HighVarPCA[:, :10])
    adata.obs_names = _tds.ca.CellID
    adata.obsm['X_pca'] = _tds.ca.HighVarPCA[:, :10]
    sc.pp.neighbors(adata, n_neighbors = 100, n_pcs = adata.obsm['X_pca'].shape[1])
    sc.tl.diffmap(adata)
    
    _tds.ca['DiffMap'] = adata.obsm['X_diffmap']
    
    diffmap1_time_corr = pd.Series(adata.obsm['X_diffmap'][:, 1], _tds.ca.CellID).corr(time_alignment_vector[tp])
    adata.uns['iroot'] = adata.obsm['X_diffmap'][:, 1].argmin() if diffmap1_time_corr>0 else adata.obsm['X_diffmap'][:, 1].argmax()
    sc.tl.dpt(adata, n_branchings=0, n_dcs=4,  min_group_size=1000)
    
    _tds.ca['Pseudotime'] = adata.obs['dpt_pseudotime'][_tds.ca.CellID].values
    _tds.ca['PseudotimeRank'] = adata.obs['dpt_pseudotime'].rank()[_tds.ca.CellID].values/len(_tds.ca.CellID)


/Users/averes/miniconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/Users/averes/miniconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


In [9]:
# for tp in range(2):
#     tds[tp].close()

In [12]:
# Move to cluster
%cp -v ../data/pseudotime/stage5.day0.loom /Volumes/adrianveres/scbeta_scrnaseq__data/
%cp -v ../data/pseudotime/stage5.day1.loom /Volumes/adrianveres/scbeta_scrnaseq__data/

../data/pseudotime/stage5.day0.loom -> /Volumes/adrianveres/scbeta_scrnaseq__data/stage5.day0.loom
../data/pseudotime/stage5.day1.loom -> /Volumes/adrianveres/scbeta_scrnaseq__data/stage5.day1.loom


In [ ]:
# Execute VGAM

sbatch -p test -n 4 -N 1 --account scrb152 --job-name s5day0 --mem 12000 --time 06:00:00 \
        --wrap """source activate py36; module load gcc/7.1.0-fasrc01 R/3.5.0-fasrc02;export R_LIBS_USER=$HOME/apps/R:$R_LIBS_USER;\
            cd /n/home15/adrianveres; python scbeta_scrnaseq/scripts/vgam_regression_wrapper.py scbeta_scrnaseq__data/stage5.day0.loom"""
        
sbatch -p test -n 4 -N 1 --account scrb152 --job-name s5day1 --mem 12000 --time 06:00:00 \
        --wrap """source activate py36; module load gcc/7.1.0-fasrc01 R/3.5.0-fasrc02;export R_LIBS_USER=$HOME/apps/R:$R_LIBS_USER;\
            cd /n/home15/adrianveres; python scbeta_scrnaseq/scripts/vgam_regression_wrapper.py scbeta_scrnaseq__data/stage5.day1.loom"""

In [13]:
%cp -v /Volumes/adrianveres/scbeta_scrnaseq__data/stage5.day0.loom ../data/pseudotime/ 
%cp -v /Volumes/adrianveres/scbeta_scrnaseq__data/stage5.day1.loom ../data/pseudotime/

/Volumes/adrianveres/scbeta_scrnaseq__data/stage5.day0.loom -> ../data/pseudotime/stage5.day0.loom
/Volumes/adrianveres/scbeta_scrnaseq__data/stage5.day1.loom -> ../data/pseudotime/stage5.day1.loom


In [3]:

tds = {}
from scbeta_scrnaseq.pseudotime import annotate_vgam_ds
for tp in range(2):
    tds[tp] = loompy.connect(f'../data/pseudotime/stage5.day{tp}.loom')
    tds[tp].vals = sp.sparse.csr_matrix(tds[tp].layers[""][:, :])
    _tds = tds[tp]
    annotate_vgam_ds(_tds)